Guillermo Blanco Núñez <br>
Pablo Díaz Blanco<br>
ap-2526-p3-ap-11-01


# Práctica 3: RNNs

## Objetivo

Los objetivos de la práctica son:
1. Diseñar y entrenar una red neuronal residual lineal para un problema de clasificación binaria.
2. Implementar un método de fine-tuning eficiente que permita adaptar el modelo a una variante del
problema sin volver a entrenar todos los parámetros.
3. Analizar las ventajas y limitaciones del método

## Preparación del dataset


Importa todas las librerías necesarias.

In [12]:
import os
import pickle
import numpy as np
from sklearn.model_selection import train_test_split


Función para cargar archivos .pkl (pickle).

In [13]:
def load_pkl(path):
    with open(path, "rb") as f:
        return pickle.load(f)

Importa el split del dataset Higgs, descargado del aula virtual.

In [14]:
DATA_PATH = "../Higgs_DS/"

train_file = os.path.join(DATA_PATH, "higgs_train.pkl")
test_file  = os.path.join(DATA_PATH, "higgs_test.pkl")
extra_file = os.path.join(DATA_PATH, "higgs_extra.pkl")



train_data = load_pkl(train_file)
test_data  = load_pkl(test_file)
extra_data = load_pkl(extra_file)


Comprueba que el conjunto de datos esté correctamente importado al mostrar por pantalla esta información del mismo.

In [15]:
print("Train shape:", train_data.shape)
print("Test shape:", test_data.shape)
print("Extra shape:", extra_data.shape)

Train shape: (2000000, 29)
Test shape: (500000, 29)
Extra shape: (2000000, 29)


Convertir a arrays de numpy y separar los datos de las etiquetas

In [16]:
train_np = np.asarray(train_data)
test_np  = np.asarray(test_data)

X = train_np[:, :-1].astype(np.float32)
y = train_np[:, -1].astype(np.float32)

X_test = test_np[:, :-1].astype(np.float32)
y_test = test_np[:, -1].astype(np.float32)


Creación de train y validation

In [17]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    shuffle=True
)

Calculo de la media y la desviación típica y aplicar normalización 

In [18]:
mean_train = X_train.mean(axis=0)
std_train  = X_train.std(axis=0)

std_train_safe = np.where(std_train == 0, 1.0, std_train)

X_train_norm = (X_train - mean_train) / std_train_safe
X_val_norm   = (X_val   - mean_train) / std_train_safe
X_test_norm  = (X_test  - mean_train) / std_train_safe


Comprobación:

In [20]:
print("Train:", X_train_norm.shape, y_train.shape)
print("Val:  ", X_val_norm.shape,   y_val.shape)
print("Test: ", X_test_norm.shape,  y_test.shape)

print("\nMedia primeras 5 features del TRAIN normalizado:")
print(X_train_norm.mean(axis=0)[:3])

print("\nSTD primeras 5 features del TRAIN normalizado:")
print(X_train_norm.std(axis=0)[:3])


Train: (1600000, 28) (1600000,)
Val:   (400000, 28) (400000,)
Test:  (500000, 28) (500000,)

Media primeras 5 features del TRAIN normalizado:
[ 1.1567707e-06 -2.7320066e-05  4.5669823e-09]

STD primeras 5 features del TRAIN normalizado:
[0.9983659 0.9999674 1.0000007]
